##  Using a pipeline to replace a mask from a sequence

In [1]:
from transformers import pipeline
nlp = pipeline("fill-mask")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from pprint import pprint
pprint(nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."))

[{'score': 0.1792745292186737,
  'sequence': '<s>HuggingFace is creating a tool that the community uses to '
              'solve NLP tasks.</s>',
  'token': 3944,
  'token_str': 'Ġtool'},
 {'score': 0.11349418759346008,
  'sequence': '<s>HuggingFace is creating a framework that the community uses '
              'to solve NLP tasks.</s>',
  'token': 7208,
  'token_str': 'Ġframework'},
 {'score': 0.052435532212257385,
  'sequence': '<s>HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.</s>',
  'token': 5560,
  'token_str': 'Ġlibrary'},
 {'score': 0.03493548929691315,
  'sequence': '<s>HuggingFace is creating a database that the community uses '
              'to solve NLP tasks.</s>',
  'token': 8503,
  'token_str': 'Ġdatabase'},
 {'score': 0.02860238403081894,
  'sequence': '<s>HuggingFace is creating a prototype that the community uses '
              'to solve NLP tasks.</s>',
  'token': 17715,
  'token_str': 'Ġprototype'}]


## Using a model and a tokenizer

In [3]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelWithLMHead.from_pretrained("distilbert-base-cased", return_dict=True)

/Users/nattiya/miniconda3/envs/dl/lib/python3.8/site-packages/transformers/modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [5]:
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."
input = tokenizer.encode(sequence, return_tensors="pt")
print("input: ", input)

decoded_sequence = tokenizer.decode(input[0])
print("decoded_sequence: ", decoded_sequence)

mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
print(torch.where(input == tokenizer.mask_token_id))

input:  tensor([[  101, 12120,  2050,  8683,  1181,  3584,  1132,  2964,  1190,  1103,
          3584,  1152, 27180,   119,  7993,  1172,  1939,  1104,  1103,  1415,
          3827,  1156,  1494,   103,  1412,  6302,  2555, 10988,   119,   102]])
decoded_sequence:  [CLS] Distilled models are smaller than the models they mimic. Using them instead of the large versions would help [MASK] our carbon footprint. [SEP]
(tensor([0]), tensor([23]))


In [6]:
output = model(input)
print("output: ", output)

token_logits = output.logits
print("token_logits: ", token_logits)

# Retrieve the predictions at the index of the mask token: this tensor has the same size as the vocabulary, 
# and the values are the scores attributed to each token. The model gives higher score to tokens it deems probable in that context.
mask_token_logits = token_logits[0, mask_token_index, :]

# Retrieve the top 5 tokens using the PyTorch topk
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

output:  MaskedLMOutput(loss=None, logits=tensor([[[ -6.6732,  -6.6450,  -6.7923,  ...,  -5.5930,  -5.2783,  -5.6559],
         [ -6.3221,  -5.6379,  -5.8990,  ...,  -4.6864,  -4.1499,  -5.3507],
         [ -5.9863,  -6.0991,  -5.8089,  ...,  -5.2297,  -4.3015,  -6.5971],
         ...,
         [ -7.8892,  -7.6718,  -7.6357,  ...,  -6.9083,  -5.5853,  -6.2459],
         [-14.7710, -14.2714, -14.1642,  ..., -11.4770, -12.1692, -13.1041],
         [-14.3695, -13.9839, -13.6330,  ..., -11.2066, -11.6754, -12.7083]]],
       grad_fn=<AddBackward0>), hidden_states=None, attentions=None)
token_logits:  tensor([[[ -6.6732,  -6.6450,  -6.7923,  ...,  -5.5930,  -5.2783,  -5.6559],
         [ -6.3221,  -5.6379,  -5.8990,  ...,  -4.6864,  -4.1499,  -5.3507],
         [ -5.9863,  -6.0991,  -5.8089,  ...,  -5.2297,  -4.3015,  -6.5971],
         ...,
         [ -7.8892,  -7.6718,  -7.6357,  ...,  -6.9083,  -5.5853,  -6.2459],
         [-14.7710, -14.2714, -14.1642,  ..., -11.4770, -12.1692, -13.1041

In [7]:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.
